# Benchmark Overview
Interaktives Notebook zur Analyse von Benchmark-Ergebnissen nach demografischen Merkmalen.

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd()
analysis_src = project_root / 'apps' / 'analysis' / 'src'
if str(analysis_src) not in sys.path: sys.path.append(str(analysis_src))

from analysis.benchmarks.analytics import BenchQuery, load_benchmark_dataframe, plot_rating_distribution, plot_rating_distribution_by_genid, plot_category_means, plot_deltas_with_significance
from analysis.persona.analytics import set_default_theme
import matplotlib.pyplot as plt
set_default_theme()


In [ ]:
import ipywidgets as W
gen_ids = W.Text(value='12,13', description='gen_ids')
models = W.Text(value='', description='models')
questions = W.Text(value='', description='questions')
alpha = W.FloatSlider(value=0.05, min=0.001, max=0.2, step=0.001, description='alpha')
perms = W.IntSlider(value=2000, min=200, max=10000, step=200, description='permutations')
display(gen_ids, models, questions, alpha, perms)

def parse_csv(text):
    txt = text.strip()
    return [t.strip() for t in txt.split(',') if t.strip()] or None

def run():
    gids = [int(x) for x in parse_csv(gen_ids.value) or []]
    mq = parse_csv(models.value)
    qq = parse_csv(questions.value)
    cfg = BenchQuery(gen_ids=gids, model_names=mq, question_uuids=qq)
    df = load_benchmark_dataframe(cfg)
    if df.empty:
        print('Keine Daten für die Auswahl.')
        return df
    display(df.head())
    ax = plot_rating_distribution(df); plt.show()
    if len(df['gen_id'].unique())>1:
        ax2 = plot_rating_distribution_by_genid(df); plt.show()
    for col in ['gender','origin_region','religion','migration_status','sexuality','marriage_status','education']:
        if col in df.columns:
            ax3 = plot_deltas_with_significance(df, col, n_perm=perms.value, alpha=alpha.value); plt.show()
    return df

btn = W.Button(description='Run analysis')
out = W.Output()
def on_click(_):
    out.clear_output()
    with out: run()
btn.on_click(on_click)
display(btn, out)


Text(value='12,13', description='gen_ids')

Text(value='', description='models')

Text(value='', description='questions')

FloatSlider(value=0.05, description='alpha', max=0.2, min=0.001, step=0.001)

IntSlider(value=2000, description='permutations', max=10000, min=200, step=200)

Button(description='Run analysis', style=ButtonStyle())

Output()